In [9]:
import pandas as pd
from pathlib import Path

In [16]:
model = 'gemma2-2b-it'
dist_folder = 'no-distil-test'
filename = 'FINAL_Completed(0 - 1319).csv'

In [17]:
basepath = (Path() / '..' /'..'/ 'logs' / model / dist_folder).resolve()

In [18]:
df = pd.read_csv(basepath / filename)

In [19]:
ex_df = pd.read_csv((Path() / '..' / '..' / 'logs' / 'gemma2b-it' / dist_folder / 'FINAL_RESULTS.csv').resolve())

In [20]:
df['extracted_answer'] = ex_df['extracted_answer']

In [7]:
for i, row in df.iterrows():
    df.loc[i, 'model_answer'] = row['model_answer'].removeprefix(row['question'])
    df.loc[i, 'model_answer'] = df.loc[i, 'model_answer'].removeprefix('\n Please respond with just the answer. The answer is: ')
    # df.loc[i, 'extracted_answer'] = row['answer'].split('####')[1].strip()

In [8]:
import re
df['model_ex1'] = df['model_answer']
for i, row in df.iterrows():
    extr = re.findall(r'[-+]?[$]?([,\d]+\.?[,\d]*)\%?(\ |$|\n){1}', row['model_answer'])
    if len(extr) != 1:  
        df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])
        continue
    v = re.sub(r',', '', extr[0][0])
    try:
        df.loc[i, 'model_ex1'] = float(v)
    except:
        df.loc[i, 'model_ex1'] = v
    df.loc[i, 'model_ex1_t'] = type(df['model_ex1'][i])

In [10]:
df['model_ex1_t'].value_counts()

model_ex1_t
<class 'float'>    1284
<class 'str'>        35
Name: count, dtype: int64

In [11]:
def save_df(df : pd.DataFrame, model : str, dist_folder : str, edition : str):
    base_path = Path()
    base_path = base_path / '..' /'..'/ 'logs' / model / dist_folder / f'{edition}.csv'
    df.to_csv(base_path)

In [ ]:
df_false = df[df['model_ex1_t'] == str]
df_false['model_ex1'].head(10)

In [13]:

for i, row in df[df['model_ex1_t'] != float].iterrows():
    inp = input(f'{i} : What is the extracted answer : {row["model_ex1"]}')
    try:
        df.loc[i, 'model_ex1'] = float(inp)
    except:
        print(f'{i} could not be parsed')

95 could not be parsed
171 could not be parsed
243 could not be parsed
429 could not be parsed
464 could not be parsed
489 could not be parsed
505 could not be parsed
509 could not be parsed
593 could not be parsed
649 could not be parsed
726 could not be parsed
744 could not be parsed
753 could not be parsed
844 could not be parsed
1017 could not be parsed
1094 could not be parsed
1159 could not be parsed
1200 could not be parsed


In [14]:
for i, row in df.iterrows():
    df.loc[i, 'model_ex1_t'] = type(row['model_ex1'])


In [18]:
df['re_query'] = False
for i, row in df[df['model_ex1_t'] != float].iterrows():
    if not row['model_answer']:
        df.loc[i, 're_query'] = True
    else:
        inp = input(f'Should this be re-queried: {row['model_answer']}')
        if not inp:
            df.loc[i, 're_query'] = True
        else:
            df.loc[i, 're_query'] = False
    

In [20]:
for i, row in df[df['model_ex1_t'] != float].iterrows():
    if not row['re_query']:
        inp = input(f'{i} : What is the extracted answer : {row["model_ex1"]}. QUESTION: {row['question']}. IDEAL_ANSWER: {row['extracted_answer']}')
        try:
            df.loc[i, 'model_ex1'] = float(inp)
        except:
            print(f'{i} could not be parsed')

In [22]:
df[df['re_query'] == True].index

Index([95, 171, 243, 429, 464, 489, 505, 593, 726, 1094, 1159, 1200], dtype='int64')

In [15]:
for i, row in df[df['model_ex1_t'] != float].iterrows():
    print(f'Model Answer: {row['model_ex1']}')
    print(f'True Answer: {row['answer']}')
    print(f'Question: {row['question']}')
    print()
    inp = input(f'{i} : What is the extracted answer : {row["model_ex1"]}')
    try:
        df.loc[i, 'model_ex1'] = float(inp)
    except:
        print(f'{i} could not be parsed')
    df.loc[i, 'model_ex1_t'] = type(row['model_ex1'])
    

Model Answer: 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 

True Answer: There are 200 x 2/5 = <<200*2/5=80>>80 boys in Grade 5.
So, 200 - 80 = <<200-80=120>>120 are girls.
Out of the 120 girls, 120 x 2/3 = <<120*2/3=80>>80 are in the girl scout.
Therefore, 120 - 80 = <<120-80=40>>40 girls are not in the girl scout.
#### 40
Question: Out of the 200 Grade 5 students, 2/5 are boys and 2/3 of the girls are in the girl scout. How many girls are not in the girl scout?

95 could not be parsed
Model Answer: 
 
 80 * 8 + 90 * 6 + 3 * 10 = 
 
 
 
 
 
 
 
 
 
 
 
 
True Answer: First find the total number of spider legs: 80 spiders * 8 legs/spider = <<80*8=640>>640 legs
Then find the total number of insect legs: 90 spiders * 6 legs/spider = <<90*6=540>>540 legs
Then find the total number of mutant invertebrate legs: 3 mutants * 10 legs/mutant = <<3*10=30>>30 legs
Then add the number of each kind of leg to find the total he saw: 640 legs + 540 legs + 30 legs = <<640+540+30=1210>>1210 legs
###

KeyboardInterrupt: Interrupted by user

In [16]:
df['correct'] = df['model_ex1'] == df['extracted_answer']

In [17]:
df['correct'].value_counts()

correct
False    1240
True       79
Name: count, dtype: int64

In [23]:
save_df(df, 'gemma2-2b-it', 'no-distil-test', 'EX1_RESULTS')